In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import lmdb
import pickle
%matplotlib inline

In [106]:


exp = dict(
    source = 'img',
    name = 'raw l:conv4 c:conv5',
    batch_size = 400,
    node = '1',
    transfer = 'conv4',
    lock = 'conv4',
    clean = 'conv5',
    data = dict(a=4)
)

exp_serial = pickle.dumps(exp)
exp2 = pickle.loads(exp_serial)


In [111]:
def create_id(exp):
    tmp = []
    tmp.append(exp.get('source',''))
    tmp.append("t:"+str(exp.get('transfer','')))
    tmp.append("l:"+str(exp.get('lock','')))
    tmp.append("c:"+str(exp.get('clean','')))
    tmp.append("n:"+str(exp.get('node','')))
    tmp.append("bs:"+str(exp.get('batch_size','')))
    return "_".join(tmp)

In [112]:
print create_id(exp)

img_t:conv4_l:conv4_c:conv5_n:1_bs:400


In [29]:
db_path = "results_lmdb"


env = lmdb.open(db_path)

with env.begin(write=True) as txn:
    txn.replace


# conn = sqlite3.connect(db_path)
# c = conn.cursor()
# c.execute("DROP TABLE IF EXISTS experiments")
# c.execute("""
# CREATE TABLE IF NOT EXISTS experiments (
# id INTEGER PRIMARY KEY, 
# source TEXT,
# layer_config TEXT,
# batch_size INTEGER,
# computer TEXT
# )""")

pass